Given a folder of images, creates a proper CSV dataset. Info about lat, long, elevation

In [ ]:
# Import necessary packages
import os
import csv
import pandas as pd

from google.colab import drive, auth
auth.authenticate_user()

In [ ]:
# Load the Drive helper and mount

# This will prompt for authorization.
drive.mount('/content/drive')

PROJECT_PATH = "/content/drive/MyDrive/Projects/Harvest Pile/ethiopia/skysat dataset" #jonathan
# PROJECT_PATH = "/content/drive/MyDrive/ethiopia/skysat dataset" #jonathan 2
# PROJECT_PATH = "/content/drive/MyDrive/ethiopia/skysat dataset" #richard

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'earth-sys262cs325b'
!gcloud config set project {project_id}
!gsutil ls


Updated property [core/project].
gs://es262-brazil-trafficking/
gs://es262-buildings_sentinel/
gs://es262-harvest-piles/
gs://es262-kilns-gen/
gs://es262-models/
gs://es262-orchards-forests/
gs://es262-pop-density/
gs://es262-poverty/
gs://es262-roadtype/
gs://open_street_maps_data/
gs://sustainlab-common/
gs://sustainlab-dhs/
gs://sustainlab-poverty-export/


In [ ]:
# Mount gcp directory to google colab - Richard
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list

!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get update

!sudo apt-get install gcsfuse

!mkdir "/content/drive/MyDrive/ethiopia/skysat dataset/amhara_skysat_all_clip_gcp"

!gcsfuse --only-dir amhara_skysat_all_clip/ es262-harvest-piles "/content/drive/MyDrive/ethiopia/skysat dataset/amhara_skysat_all_clip_gcp"



deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2659  100  2659    0     0  10138      0 --:--:-- --:--:-- --:--:-- 10148
OK
Get:1 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5,004 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcon

In [ ]:
len(os.listdir("/content/drive/MyDrive/ethiopia/skysat dataset/amhara_skysat_all_clip_gcp"))

75874

# Create CSV

In [ ]:
!pip install pyproj

In [ ]:
from osgeo import gdal

In [ ]:
# Get image coordinates
def get_coordinates(f):
  ds = gdal.Open(f)
  gt = ds.GetGeoTransform()

  # print(ds.GetProjection())
  width = ds.RasterXSize
  height = ds.RasterYSize
  # print(width, height)

  minx = gt[0]
  miny = gt[3] + width*gt[4] + height*gt[5]
  maxx = gt[0] + width*gt[1] + height*gt[2]
  maxy = gt[3]

  # Upper-left
  lon1, lat1 = minx, maxy
  # Bottom-right
  lon2, lat2 = maxx, miny

  return(lat1, lon1, lat2, lon2)

In [ ]:
df = pd.DataFrame(columns=["filename", "lat_1", "lon_1", "lat_2", "lon_2", "activity"])

In [ ]:
image_folder = os.path.join(PROJECT_PATH, "amhara_skysat_clip_train")
len(os.listdir(image_folder))

677

In [ ]:
count = 0

for filename in os.listdir(image_folder):
    f = os.path.join(image_folder, filename)
    print(f)
    break
    try:
      lat1, lon1, lat2, lon2 = get_coordinates(f)
      df = pd.concat(
          [pd.DataFrame([[os.path.join(sub_folder, filename), lat1, lon1, lat2, lon2]],
                        columns=["filename", "lat_1", "lon_1", "lat_2", "lon_2"]), df],
          ignore_index=True)
      count += 1
    except Exception as e:
      print("failed on ", f)
      print(e)

    if count % 100 == 0:
      print(count)

In [ ]:
count = 0

for index, row in df.iterrows():
  f = os.path.join(PROJECT_PATH, row["filename"])
  try:
    if not os.path.exists(f):
      file_name = row["filename"][len("amhara_skysat_all_clip/"):]
      df.at[index, "filename"] = os.path.join("amhara_skysat_clip_train", file_name)
    count += 1
  except Exception as e:
    print("failed on ", f)
    print(e)

  if count % 100 == 0:
    print(count)

In [ ]:
f = "/content/drive/MyDrive/Projects/Harvest Pile/ethiopia/skysat dataset/amhara_skysat_all_clip/20230118_071229_ssc13_u0004_visual_1058.tif"
lat1, lon1, lat2, lon2 = get_coordinates(f)

In [ ]:
print(lat1, lon1, lat2, lon2)

1357607.5 563779.5 1357351.5 564035.5


In [ ]:
import math
count = 0

for index, row in df.iterrows():
  f = os.path.join(PROJECT_PATH, "amhara_labelled", row["filename"])
  try:
    lat1, lon1, lat2, lon2 = get_coordinates(f)
    df.at[index, "lat_1"] = lat1
    df.at[index, "lon_1"] = lon1
    df.at[index, "lat_2"] = lat2
    df.at[index, "lon_2"] = lon2

  except Exception as e:
    print("failed on ", f)
    print(e)

  count += 1
  if count % 100 == 0:
    print(count)

100
200
300
400
500
600


In [ ]:
df[df['filename'].str.startswith('amhara_skysat_clip_train')]

In [ ]:
labelled_df = df[df["activity"].notna()]

In [ ]:
df

,filename,lon_1,lat_1,lon_2,lat_2,activity,lat_mean,lon_mean,year,month,day
26,20230116_072932_ss01_u0001_visual_1941.tif,38.806172,8.982793,38.808502,8.980479,0.0,992827.5,478822.5,2023,1,16
165,20221130_075418_ssc1_u0001_visual_0432.tif,38.963336,11.982696,38.965687,11.980382,1.0,1324512.5,496136.5,2022,11,30
262,20221130_075418_ssc1_u0001_visual_1740.tif,38.993906,11.964178,38.996257,11.961863,1.0,1322464.5,499464.5,2022,11,30
334,20221119_080913_ssc1_u0001_visual_0220.tif,37.292682,11.635510,37.295044,11.633210,0.0,1286681.0,313999.5,2022,11,19
358,20221119_080913_ssc1_u0001_visual_0244.tif,37.293020,11.579971,37.295381,11.577670,1.0,1280537.0,313999.5,2022,11,19
...,...,...,...,...,...,...,...,...,...,...,...
50552,20230123_071148_ssc13_u0001_visual_0140.tif,38.953812,11.974289,38.956163,11.971974,0.0,0.0,NaN,2023,1,23
50591,20221031_134804_ssc19_u0001_visual_0138.tif,38.778613,8.971915,38.780943,8.969601,0.0,0.0,NaN,2022,10,31
50612,20221031_134804_ssc19_u0001_visual_0156.tif,38.778638,8.930235,38.780968,8.927921,0.0,0.0,NaN,2022,10,31
50757,20221031_134804_ssc19_u0001_visual_0457.tif,38.785625,8.927924,38.787955,8.925610,0.0,NaN,NaN,2022,10,31


In [ ]:
import shutil

for index, row in labelled_df.iterrows():
  src_f = row["filename"]
  src_path = os.path.join(PROJECT_PATH, src_f)

  fname = src_f[src_f.index('/') + 1:]
  dest_path = os.path.join(PROJECT_PATH, "amhara_labelled", fname)

  shutil.copy(src_path, dest_path)
  labelled_df.at[index, "filename"] = fname

In [ ]:
from pyproj import Transformer

In [ ]:
# Example snippet to convert coords from df - Richard

# df = pd.DataFrame({
#     'filename': ['test'],
#     'lat_1': [1582015],
#     'lon_1': [519108.5],
#     'lat_2': [1581759],
#     'lon_2': [519364.5],
#     'activity': [1]
# })

transformer = Transformer.from_crs("EPSG:32637", "EPSG:4326")

def transform_coordinates(row):
    lat1, lon1 = transformer.transform(row['lon_1'], row['lat_1'])
    lat2, lon2 = transformer.transform(row['lon_2'], row['lat_2'])
    return pd.Series({'lat_1':lat1, 'lon_1':lon1, 'lat_2':lat2, 'lon_2':lon2})

# Apply transformation
transformed_coordinates = df.apply(transform_coordinates, axis=1)

# Overwrite original columns
df['lat_1'], df['lon_1'], df['lat_2'], df['lon_2'] = transformed_coordinates['lat_1'], transformed_coordinates['lon_1'], transformed_coordinates['lat_2'], transformed_coordinates['lon_2']


<ipython-input-48-e546bf137dcf>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat_1'], df['lon_1'], df['lat_2'], df['lon_2'] = transformed_coordinates['lat_1'], transformed_coordinates['lon_1'], transformed_coordinates['lat_2'], transformed_coordinates['lon_2']


Load pre-existing csv

In [ ]:
image_folder = os.path.join(PROJECT_PATH, "amhara_skysat_all_clip")
print(len(os.listdir(image_folder)))

In [ ]:
csv_path = os.path.join(PROJECT_PATH, "amhara_labels.csv")
df = pd.read_csv(csv_path)

In [ ]:
df = df.iloc[:, 1:]

In [ ]:
mismatched = df[df["activity"] != df["expert"]]

In [ ]:
len(mismatched)

173

In [ ]:
print(len(mismatched[mismatched["activity"] == 0]))

102


In [ ]:
df.rename(columns={'has_piles': 'activity'}, inplace=True)

In [ ]:
df.drop("model_preds", axis = 1, inplace=True)

In [ ]:
# Rename filename so it uses mounted directory
df['filename'] = df['filename'].str.replace('/', '_gcp/', n=1)

In [ ]:
df[df['lon_1'].isnull()]

In [ ]:
count = 0
for index, row in df.iterrows():
    filename = row["filename"]
    f = os.path.join(PROJECT_PATH, filename)
    try:
      lat1, lon1, lat2, lon2 = get_coordinates(f)
      df.at[index, "lat_1"] = lat1
      df.at[index, "lat_2"] = lat2
      df.at[index, "lon_1"] = lon1
      df.at[index, "lon_2"] = lon2
      count += 1
    except Exception as e:
      print("failed on ", f)
      print(e)

    if count % 100 == 0:
      print(count)

# Coordinates
https://colab.research.google.com/drive/15x1bwqMzEd6EcEDPFqArDC-OLtgdD8I8

In [ ]:
df

,filename,lon_1,lat_1,lon_2,lat_2,activity,lat_mean,lon_mean,year,month,day
0,amhara_skysat_all_clip/20230116_072932_ss01_u0...,478694.5,995259.5,478950.5,995003.5,NaN,995131.5,478822.5,2023,1,16
1,amhara_skysat_all_clip/20230116_072932_ss01_u0...,478694.5,997307.5,478950.5,997051.5,NaN,997179.5,478822.5,2023,1,16
2,amhara_skysat_all_clip/20230116_072932_ss01_u0...,478694.5,996539.5,478950.5,996283.5,NaN,996411.5,478822.5,2023,1,16
3,amhara_skysat_all_clip/20230116_072932_ss01_u0...,478694.5,995515.5,478950.5,995259.5,NaN,995387.5,478822.5,2023,1,16
4,amhara_skysat_all_clip/20230116_072932_ss01_u0...,478694.5,997051.5,478950.5,996795.5,NaN,996923.5,478822.5,2023,1,16
...,...,...,...,...,...,...,...,...,...,...,...
51099,amhara_skysat_all_clip/20230123_071148_ssc13_u...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1,23
51100,amhara_skysat_all_clip/20230123_071148_ssc13_u...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1,23
51101,amhara_skysat_all_clip/20230123_071148_ssc13_u...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1,23
51102,amhara_skysat_all_clip/20230123_071148_ssc13_u...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1,23


In [ ]:
df.rename(columns={'lat_mean': 'lon_mean', "lon_mean": "lat_mean"}, inplace=True)

In [ ]:
df["lat_mean"] = df[['lat_1', 'lat_2']].mean(axis=1)
df["lon_mean"] = df[['lon_1', 'lon_2']].mean(axis=1)

<ipython-input-52-face7378562a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lat_mean"] = df[['lat_1', 'lat_2']].mean(axis=1)
<ipython-input-52-face7378562a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lon_mean"] = df[['lon_1', 'lon_2']].mean(axis=1)


# Date

In [ ]:
df.dtypes

filename     object
lat_1       float64
lon_1       float64
lat_2       float64
lon_2       float64
activity     object
lat_mean    float64
lon_mean    float64
year          int64
month         int64
day           int64
dtype: object

In [ ]:
len("amhara_skysat_all_clip/")

23

In [ ]:
start_idx = 23
df['year'] = df['filename'].str[start_idx: start_idx+4]
df['month'] = df['filename'].str[start_idx+4: start_idx+6]
df['day'] = df['filename'].str[start_idx+6: start_idx+8]

In [ ]:
len("black_south/")

12

In [ ]:
test_str = "20220328_073642_ssc2_u0001_visual_clip/20220328_073642_ssc2_u0001_visual_clip_4915.tif"
start_idx = test_str.find("/") + 1

2022 03 28


In [ ]:
df["year"] = df["year"].astype("int")
df["month"] = df["month"].astype("int")
df["day"] = df["day"].astype("int")

# Elevation
https://colab.research.google.com/drive/19_FMqOCUjGoHmr47BMdeXd9l4YscN4Qd

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=rF8_a2uAu8CD10hDIV4rf9CGLx7Gu0uBpzoPAv6RvdY&tc=-PK8LFMTNPOhYNgF-tB4eNMvFnbJVMzKtusHLjpJ6Mk&cc=V-jCi5NwunECDiP45N_N3cskakX7GYstTdJfwWH4ziU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhVZyABRuVm6PiJrg2Uens3HSYa3I6EJKKB_XQdHohGMJQap3Dj7p_s

Successfully saved authorization token.


In [ ]:
# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')
projection = elv.projection();
scale = elv.projection().nominalScale();

In [ ]:
df["altitude"] = 0
BATCH_SIZE = 500 # Sample 500 rows at a time, fast
df_size = len(df)
counter = 0

# string_columns = df.select_dtypes(include='object').columns
# min_itemsize = {col: df_slice[col].str.len().max() + 1 for col in string_columns}
store = pd.HDFStore('output_store.h5', mode='a', min_itemsize=90)


<ipython-input-56-925ba3d4c2d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["altitude"] = 0


In [ ]:
while counter < df_size - BATCH_SIZE:
  df_slice = df.iloc[counter:counter + BATCH_SIZE]
  # Create a list of point features with ID properties

  lat_lng_list = df_slice[['lon_mean', 'lat_mean']].values.tolist() # because ee.Geometry.Point takes points at x,y
  point_list = []
  for i, coord in enumerate(lat_lng_list):
    point = ee.Geometry.Point(coord)
    feature = ee.Feature(point, {'id': i})
    point_list.append(feature)

  # Create a FeatureCollection from the point features
  fc = ee.FeatureCollection(point_list)

  # Sample the elevation at the points
  elv_points = elv.sampleRegions(collection=fc, scale=scale, projection=projection)
  # Sort the FeatureCollection based on the ID property
  elv_points = elv_points.sort('id')

  elevationList = elv_points.aggregate_array('elevation').getInfo();
  df_slice['altitude'] = elevationList
  store.append('my_data', df_slice, min_itemsize=90)

  counter += BATCH_SIZE

  if counter % BATCH_SIZE == 0:
    print(counter)

# Running last time for remaining elements that are not % 10k
df_slice = df[counter:]
df_slice["lat_mean"] = df_slice[['lat_1', 'lat_2']].mean(axis=1)
df_slice["lon_mean"] = df_slice[['lon_1', 'lon_2']].mean(axis=1)
lat_lng_list = df_slice[['lon_mean', 'lat_mean']].values.tolist()
point_list = []
for i, coord in enumerate(lat_lng_list):
  point = ee.Geometry.Point(coord)
  feature = ee.Feature(point, {'id': i})
  point_list.append(feature)

fc = ee.FeatureCollection(point_list)
elv_points = elv.sampleRegions(collection=fc, scale=scale, projection=projection)
# Sort the FeatureCollection based on the ID property
elv_points = elv_points.sort('id')

elevationList = elv_points.aggregate_array('elevation').getInfo();
df_slice['altitude'] = elevationList
store.append('my_data', df_slice)

store.close()

<ipython-input-58-429ded431a3f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice['altitude'] = elevationList
<ipython-input-58-429ded431a3f>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice["lat_mean"] = df_slice[['lat_1', 'lat_2']].mean(axis=1)
<ipython-input-58-429ded431a3f>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

500


<ipython-input-58-429ded431a3f>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice['altitude'] = elevationList


In [ ]:
# Close the HDF5 file
store.close()

# Open the HDF5 file in 'r' mode (read-only mode)
store = pd.HDFStore('output_store.h5', mode='r+')

# Read data from the file
out_df = store['my_data']

store.close()

In [ ]:
out_df

,filename,lon_1,lat_1,lon_2,lat_2,activity,lat_mean,lon_mean,year,month,day,altitude
26,20230116_072932_ss01_u0001_visual_1941.tif,38.806172,8.982793,38.808502,8.980479,0.0,8.981636,38.807337,2023,1,16,2327
165,20221130_075418_ssc1_u0001_visual_0432.tif,38.963336,11.982696,38.965687,11.980382,1.0,11.981539,38.964512,2022,11,30,1922
262,20221130_075418_ssc1_u0001_visual_1740.tif,38.993906,11.964178,38.996257,11.961863,1.0,11.963020,38.995081,2022,11,30,1975
334,20221119_080913_ssc1_u0001_visual_0220.tif,37.292682,11.635510,37.295044,11.633210,0.0,11.634360,37.293863,2022,11,19,1810
358,20221119_080913_ssc1_u0001_visual_0244.tif,37.293020,11.579971,37.295381,11.577670,1.0,11.578821,37.294201,2022,11,19,1856
...,...,...,...,...,...,...,...,...,...,...,...,...
50552,20230123_071148_ssc13_u0001_visual_0140.tif,38.953812,11.974289,38.956163,11.971974,0.0,11.973132,38.954988,2023,1,23,1913
50591,20221031_134804_ssc19_u0001_visual_0138.tif,38.778613,8.971915,38.780943,8.969601,0.0,8.970758,38.779778,2022,10,31,2298
50612,20221031_134804_ssc19_u0001_visual_0156.tif,38.778638,8.930235,38.780968,8.927921,0.0,8.929078,38.779803,2022,10,31,2128
50757,20221031_134804_ssc19_u0001_visual_0457.tif,38.785625,8.927924,38.787955,8.925610,0.0,8.926767,38.786790,2022,10,31,2148


# Save

In [ ]:
len(out_df)

689

In [ ]:
out_df.to_csv('output.csv')

In [ ]:
!cp "output.csv" "/content/drive/MyDrive/Projects/Harvest Pile/ethiopia/skysat dataset/amhara_labels.csv"